In [3]:
from allennlp.predictors.predictor import Predictor
from pprint import pprint

predictor = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/elmo-constituency-parser-2018.03.14.tar.gz")


100%|██████████| 710802195/710802195 [00:27<00:00, 25464514.85B/s]
Your label namespace was 'pos'. We recommend you use a namespace ending with 'labels' or 'tags', so we don't add UNK and PAD tokens by default to your vocabulary.  See documentation for `non_padded_namespaces` parameter in Vocabulary.


{'class_probabilities': [[0.9999688267707825,
   3.415389065314356e-12,
   2.6355936679520013e-14,
   1.4248029422958552e-08,
   2.1221376517690804e-18,
   1.077136223639509e-15,
   6.459481838305692e-10,
   8.62533795498166e-08,
   9.841613746175426e-09,
   6.982374145536596e-08,
   2.312473637870651e-14,
   6.541837384299587e-13,
   3.344712729003163e-09,
   2.2047016989823476e-10,
   9.514343091154842e-09,
   2.4894989110180177e-05,
   2.5630554604150375e-14,
   1.3189342196531784e-12,
   1.0560157193140185e-09,
   1.184306125390222e-10,
   1.0353929935646988e-09,
   4.6387981456064153e-08,
   2.2954212752779313e-08,
   1.0656828197674884e-10,
   2.6044117475976236e-06,
   1.4538519277351725e-09,
   7.281308368745076e-12,
   1.0116353905686992e-06,
   3.5680978571067667e-12,
   2.5771504930105493e-09,
   2.7017389214734067e-08,
   9.798010403017088e-09,
   2.0772224615939194e-06,
   9.13497125165641e-11,
   1.3093280626375048e-10,
   4.404047615480522e-08,
   2.3794410886068817e-09,

In [154]:
sent = '''
The American rift with Syria deepened this week, with suspicions that Syria might have been behind the assassination of Lebanon's former prime minister in Beirut on Monday.'''
trigger_text = 'former'


In [15]:
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification

root_dir = '/shared/lyuqing/probing_for_event/'
os.chdir(root_dir)

model_dir = 'output_model_dir/'
model_name = 'elior_mnli_3_yesno_format_bert-l'

te_model = AutoModelForSequenceClassification.from_pretrained(f'{model_dir}{model_name}').to('cuda:0')


RuntimeError: Error(s) in loading state_dict for BertForSequenceClassification:
	size mismatch for bert.encoder.layer.0.intermediate.dense.weight: copying a param with shape torch.Size([1024, 4096]) from checkpoint, the shape in current model is torch.Size([4096, 1024]).
	size mismatch for bert.encoder.layer.0.output.dense.weight: copying a param with shape torch.Size([4096, 1024]) from checkpoint, the shape in current model is torch.Size([1024, 4096]).
	size mismatch for bert.encoder.layer.1.intermediate.dense.weight: copying a param with shape torch.Size([1024, 4096]) from checkpoint, the shape in current model is torch.Size([4096, 1024]).
	size mismatch for bert.encoder.layer.1.output.dense.weight: copying a param with shape torch.Size([4096, 1024]) from checkpoint, the shape in current model is torch.Size([1024, 4096]).
	size mismatch for bert.encoder.layer.2.intermediate.dense.weight: copying a param with shape torch.Size([1024, 4096]) from checkpoint, the shape in current model i